# step_renderer

> Composable render functions for the review card stack step

In [ ]:
#| default_exp components.step_renderer

In [ ]:
#| export
from typing import Any, List, Optional

from fasthtml.common import Div, Span, Audio, Details, Summary

# DaisyUI components
from cjm_fasthtml_daisyui.utilities.semantic_colors import text_dui, bg_dui
from cjm_fasthtml_daisyui.components.data_display.collapse import (
    collapse, collapse_title, collapse_content, collapse_modifiers
)

# Tailwind utilities
from cjm_fasthtml_tailwind.utilities.spacing import p, m
from cjm_fasthtml_tailwind.utilities.sizing import w, h, min_h
from cjm_fasthtml_tailwind.utilities.typography import font_size, font_weight
from cjm_fasthtml_tailwind.utilities.layout import overflow, display_tw
from cjm_fasthtml_tailwind.utilities.flexbox_and_grid import (
    flex_display, flex_direction, justify, items, gap, grow
)
from cjm_fasthtml_tailwind.core.base import combine_classes

# Keyboard navigation
from cjm_fasthtml_keyboard_navigation.components.system import render_keyboard_system
from cjm_fasthtml_keyboard_navigation.components.hints import render_keyboard_hints

# Card stack library
from cjm_fasthtml_card_stack.components.viewport import render_viewport
from cjm_fasthtml_card_stack.components.controls import render_card_count_select
from cjm_fasthtml_card_stack.components.progress import render_progress_indicator
from cjm_fasthtml_card_stack.core.models import CardStackState
from cjm_fasthtml_card_stack.core.constants import DEFAULT_VISIBLE_COUNT, DEFAULT_CARD_WIDTH
from cjm_fasthtml_card_stack.keyboard.actions import (
    render_card_stack_action_buttons, build_card_stack_url_map
)

# Review configuration
from cjm_transcript_review.components.card_stack_config import (
    REVIEW_CS_CONFIG, REVIEW_CS_IDS, REVIEW_CS_BTN_IDS,
)
from cjm_transcript_review.components.keyboard_config import create_review_keyboard_manager

# Local imports
from cjm_transcript_review.html_ids import ReviewHtmlIds
from cjm_transcript_review.models import ReviewUrls
from cjm_transcript_review.components.review_card import (
    create_review_card_renderer, AssembledSegment
)
from cjm_transcript_review.components.callbacks import generate_review_callbacks_script
from cjm_transcript_review.components.audio_controls import render_audio_controls

# Debug flag
DEBUG_REVIEW_RENDER = False

## Toolbar & Stats

Review toolbar with card count selector and statistics.

In [ ]:
#| export
def render_review_toolbar(
    visible_count:int=DEFAULT_VISIBLE_COUNT,  # Current visible card count
    is_auto_mode:bool=False,  # Whether card count is in auto-adjust mode
    playback_speed:float=1.0,  # Current playback speed
    auto_navigate:bool=False,  # Whether auto-navigate is enabled
    urls:ReviewUrls=None,  # URL bundle for audio control routes
    oob:bool=False,  # Whether to render as OOB swap
) -> Any:  # Toolbar component
    """Render the review toolbar with audio controls and card count selector."""
    urls = urls or ReviewUrls()
    
    return Div(
        # Left: Audio controls (speed + auto-navigate)
        render_audio_controls(
            current_speed=playback_speed,
            auto_navigate=auto_navigate,
            speed_url=urls.speed_change,
            auto_nav_url=urls.toggle_auto_nav,
        ),
        
        # Spacer
        Div(cls=str(grow())),

        # Right: Card count selector
        Div(
            Span(
                "Cards:",
                cls=combine_classes(font_size.sm, text_dui.base_content.opacity(70), m.r(2))
            ),
            render_card_count_select(
                REVIEW_CS_CONFIG, REVIEW_CS_IDS,
                current_count=visible_count,
                is_auto_mode=is_auto_mode,
            ),
            cls=combine_classes(flex_display, items.center)
        ),

        id=ReviewHtmlIds.REVIEW_TOOLBAR,
        cls=combine_classes(flex_display, gap(2), items.center),
        hx_swap_oob="true" if oob else None
    )

In [ ]:
#| export
def render_review_stats(
    assembled:List[AssembledSegment],  # Assembled segments
    oob:bool=False,  # Whether to render as OOB swap
) -> Any:  # Statistics component
    """Render review statistics."""
    total = len(assembled)
    total_dur = sum(a.vad_chunk.duration for a in assembled) if assembled else 0.0

    return Div(
        Span(
            f"{total} segments \u00b7 {total_dur:.1f}s total",
            cls=combine_classes(font_size.sm, text_dui.base_content.opacity(70))
        ),
        id=ReviewHtmlIds.REVIEW_STATS,
        hx_swap_oob="true" if oob else None
    )

## Keyboard Hints

Collapsible keyboard shortcut hints for the review step.

In [ ]:
#| export
def render_review_keyboard_hints(
    oob:bool=False,  # Whether to render as OOB swap
) -> Any:  # Collapsible keyboard hints component
    """Render keyboard shortcut hints in a collapsible container."""
    # Create keyboard manager to extract hints
    kb_manager = create_review_keyboard_manager(
        ids=REVIEW_CS_IDS,
        button_ids=REVIEW_CS_BTN_IDS,
        config=REVIEW_CS_CONFIG,
    )
    
    hints = render_keyboard_hints(
        kb_manager,
        include_navigation=True,
        include_zone_switch=False,  # Single zone, no switching
        badge_style="outline",
        container_id="review-kb-hints",
        use_icons=False
    )
    
    return Details(
        Summary(
            "Keyboard Shortcuts",
            cls=combine_classes(collapse_title, font_size.sm, font_weight.medium)
        ),
        Div(
            hints,
            cls=collapse_content
        ),
        id=ReviewHtmlIds.KEYBOARD_HINTS,
        cls=combine_classes(collapse, collapse_modifiers.arrow, bg_dui.base_200),
        hx_swap_oob="true" if oob else None
    )

## Main Content Area

Renders the review content area with card stack viewport and audio element.

In [ ]:
#| export
def render_review_content(
    assembled:List[AssembledSegment],  # Assembled segments to display
    focused_index:int,  # Currently focused segment index
    visible_count:int,  # Number of visible cards in viewport
    card_width:int,  # Card stack width in rem
    urls:ReviewUrls,  # URL bundle for review routes
    media_path:Optional[str]=None,  # Path to audio file for playback
) -> Any:  # Main content area
    """Render the review content area with card stack viewport and keyboard system."""
    if DEBUG_REVIEW_RENDER:
        print(f"[REVIEW_RENDER] render_review_content called")
        print(f"[REVIEW_RENDER] assembled count: {len(assembled)}")
        print(f"[REVIEW_RENDER] media_path: {media_path}")

    # Create card renderer callback
    card_renderer = create_review_card_renderer()

    # Build CardStackState for library viewport
    cs_state = CardStackState(
        focused_index=focused_index,
        visible_count=visible_count,
        card_width=card_width,
    )

    # Render viewport from library
    viewport = render_viewport(
        card_items=assembled,
        state=cs_state,
        config=REVIEW_CS_CONFIG,
        ids=REVIEW_CS_IDS,
        urls=urls.card_stack,
        render_card=card_renderer,
        form_input_name="segment_index",
    )

    # Build audio source URL
    audio_src = ""
    if urls.audio_src and media_path:
        audio_src = f"{urls.audio_src}?path={media_path}"

    # Generate card stack JavaScript callbacks with audio playback
    callbacks_script = generate_review_callbacks_script(
        ids=REVIEW_CS_IDS,
        button_ids=REVIEW_CS_BTN_IDS,
        config=REVIEW_CS_CONFIG,
        urls=urls.card_stack,
        container_id=ReviewHtmlIds.REVIEW_CONTENT,
        focus_input_id=REVIEW_CS_IDS.focused_index_input,
        audio_player_id=ReviewHtmlIds.AUDIO_PLAYER,
    )

    # Build keyboard system internally
    kb_manager = create_review_keyboard_manager(
        ids=REVIEW_CS_IDS,
        button_ids=REVIEW_CS_BTN_IDS,
        config=REVIEW_CS_CONFIG,
    )
    
    # URL mappings (card stack navigation)
    include_selector = f"#{REVIEW_CS_IDS.focused_index_input}"
    url_map = build_card_stack_url_map(REVIEW_CS_BTN_IDS, urls.card_stack)
    
    # Target, include, and swap maps
    target = f"#{REVIEW_CS_IDS.card_stack}"
    target_map = {btn_id: target for btn_id in url_map}
    include_map = {btn_id: include_selector for btn_id in url_map}
    swap_map = {btn_id: "none" for btn_id in url_map}  # OOB swaps handle updates
    
    kb_system = render_keyboard_system(
        kb_manager,
        url_map=url_map,
        target_map=target_map,
        include_map=include_map,
        swap_map=swap_map,
        show_hints=False,  # Hints rendered separately
        include_state_inputs=True,
    )

    return Div(
        # Card stack viewport
        viewport,

        # Keyboard navigation system
        kb_system.script,
        kb_system.hidden_inputs,
        kb_system.action_buttons,

        # Hidden audio element for audition playback
        Audio(
            id=ReviewHtmlIds.AUDIO_PLAYER,
            src=audio_src,
            preload="metadata",
            cls=str(display_tw.hidden),
        ),

        # Hidden action buttons for JS-triggered card stack nav
        render_card_stack_action_buttons(REVIEW_CS_BTN_IDS, urls.card_stack, REVIEW_CS_IDS),

        # JavaScript callbacks (card stack JS + audio playback)
        callbacks_script,

        id=ReviewHtmlIds.REVIEW_CONTENT,
        cls=combine_classes(grow(), min_h(0), overflow.hidden, flex_display, flex_direction.col)
    )

## Footer

Footer with progress indicator and statistics.

In [ ]:
#| export
def render_review_footer(
    assembled:List[AssembledSegment],  # Assembled segments
    focused_index:int,  # Currently focused segment index
) -> Any:  # Footer content with progress indicator and stats
    """Render footer content with progress indicator and statistics."""
    total_segments = len(assembled)

    return Div(
        render_progress_indicator(focused_index, total_segments, REVIEW_CS_IDS, label="Segment"),
        render_review_stats(assembled),
        id=ReviewHtmlIds.REVIEW_FOOTER,
        cls=combine_classes(flex_display, justify.between, items.center, gap(4))
    )

## Full Step Renderer

Renders the complete review step with toolbar, content, and footer.

In [ ]:
#| export
def render_review_step(
    assembled:List[AssembledSegment],  # Assembled segments to display
    focused_index:int=0,  # Currently focused segment index
    visible_count:int=DEFAULT_VISIBLE_COUNT,  # Number of visible cards
    is_auto_mode:bool=False,  # Whether card count is in auto-adjust mode
    card_width:int=DEFAULT_CARD_WIDTH,  # Card stack width in rem
    playback_speed:float=1.0,  # Current playback speed
    auto_navigate:bool=False,  # Whether auto-navigate is enabled
    urls:ReviewUrls=None,  # URL bundle for review routes
    media_path:Optional[str]=None,  # Path to audio file for playback
) -> Any:  # Complete review step component
    """Render the complete review step with toolbar, content, and footer."""
    urls = urls or ReviewUrls()
    
    return Div(
        # Keyboard hints (collapsible)
        render_review_keyboard_hints(),
        
        # Toolbar with audio controls
        render_review_toolbar(
            visible_count, is_auto_mode,
            playback_speed=playback_speed,
            auto_navigate=auto_navigate,
            urls=urls,
        ),
        
        # Main content area (includes keyboard system internally)
        render_review_content(
            assembled, focused_index, visible_count, card_width, urls, media_path,
        ),
        
        # Footer
        render_review_footer(assembled, focused_index),
        
        id=ReviewHtmlIds.REVIEW_CONTAINER,
        cls=combine_classes(
            flex_display, flex_direction.col, gap(4),
            h.full, p(4)
        )
    )

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()